In [1]:
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, ConcatDataset, WeightedRandomSampler, Subset, DataLoader
import os
import torchxrayvision as xrv
import torchvision.transforms as transforms
from skimage.color import rgb2gray
from skimage.transform import resize
import pydicom
from torchxrayvision.datasets import XRayCenterCrop
import pandas as pd
import wandb
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, classification_report, confusion_matrix
import helpers, train_utils, classes
from collections import Counter
import torch
import torch.nn as nn

c:\Users\user-pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
dicom_dir_1 = 'C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 1'
metadata_1 = pd.read_excel('C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 1/FileDatabaseWithRadiology.xlsx')
dicom_dir_2 = 'C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 2'
metadata_2 = pd.read_excel('C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 2/Database_Training-2024.08.28.xlsx')

d1 = classes.DICOMDataset1(dicom_dir=dicom_dir_1, metadata_df=metadata_1, target_size=224) 
d2 = classes.DICOMDataset2(dicom_dir=dicom_dir_2, metadata_df=metadata_2, target_size=224)

# Split datasets and store indices
train_indices_d1, val_indices_d1, test_indices_d1 = helpers.split_dataset(d1)
train_indices_d2, val_indices_d2, test_indices_d2 = helpers.split_dataset(d2)

# Save indices for later use
split_indices = {
    'd1': {'train': train_indices_d1, 'val': val_indices_d1, 'test': test_indices_d1},
    'd2': {'train': train_indices_d2, 'val': val_indices_d2, 'test': test_indices_d2}
}

label = 'Profusion'
d1.set_target(target_label=label, target_size=224)
d2.set_target(target_label=label, target_size=224)

train_d1 = Subset(d1, train_indices_d1)
val_d1 = Subset(d1, val_indices_d1)
test_d1 = Subset(d1, test_indices_d1)

train_d2 = Subset(d2, train_indices_d2)
val_d2 = Subset(d2, val_indices_d2)
test_d2 = Subset(d2, test_indices_d2)

In [3]:
def create_weighted_sampler(dataset, target_label):
    # Calculate class weights
    class_counts = np.bincount([label for _, label in dataset])
    class_weights = 1. / class_counts
    sample_weights = [class_weights[label] for _, label in dataset]

    # Create a weighted sampler
    sampler = WeightedRandomSampler(sample_weights, len(sample_weights))
    return sampler

# Create the base datasets
train_d1 = Subset(d1, train_indices_d1)
train_d2 = Subset(d2, train_indices_d2)

# Define augmentations
augmentations_list = [
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.RandomRotation(15),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))
]

# Create augmented datasets
augmented_train_d1 = classes.AugmentedDataset(base_dataset=train_d1, augmentations_list=augmentations_list)
augmented_train_d2 = classes.AugmentedDataset(base_dataset=train_d2, augmentations_list=augmentations_list)

    # Create dataloaders
train_loader_d1, train_aug_loader_d1, val_loader_d1, test_loader_d1 = helpers.create_dataloaders(
    train_d1, augmented_train_d1, val_d1, test_d1, batch_size=32, target=label
)

train_loader_d2, train_aug_loader_d2, val_loader_d2, test_loader_d2 = helpers.create_dataloaders(
    train_d2, augmented_train_d2, val_d2, test_d2, batch_size=32, target=label
)

LENGTHS: (824, 3296)
LENGTHS: (26, 103)
LENGTHS: (599, 2396)
LENGTHS: (19, 75)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = xrv.models.DenseNet(weights="densenet121-res224-all").to(device)
model.classifier = classes.BaseClassifier(in_features=1024
                                          
                                          )
augmentations = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))
])


In [5]:
target_label = 'Profusion Label'

print(train_d1.dataset.metadata_df[target_label].value_counts())

1.0    814
0.0    364
Name: Profusion Label, dtype: int64


In [6]:
def get_alpha_FLoss(train, target_label):
    """
    Compute a single alpha value for binary focal loss.
    
    """
    class_counts = train.dataset.metadata_df[target_label].value_counts()
    class_counts = class_counts.sort_index()

    # Ensure class counts exist
    if len(class_counts) < 2:
        raise ValueError("Dataset must contain both positive (1) and negative (0) samples.")

    minority = class_counts[0]  # Assuming class 0 is the minority
    majority = class_counts[1]  # Assuming class 1 is the majority

    # Compute alpha as the proportion of the negative class
    alpha = minority / (majority + minority)

    return np.round(alpha, 2)

In [7]:
print(get_alpha_FLoss(train_d1, 'Profusion Label'))

0.31


In [8]:
def compute_pos_weight(train, target_label):
    """Compute pos_weight for BCEWithLogitsLoss."""
    class_counts = train.dataset.metadata_df[target_label].value_counts()
    class_counts = class_counts.sort_index()

    if len(class_counts) < 2:
        raise ValueError("Dataset must contain both positive (1) and negative (0) samples.")

    N_pos = class_counts[1]  # Positive class count
    N_neg = class_counts[0]  # Negative class count

    pos_weight = torch.tensor([N_neg / N_pos], dtype=torch.float32)  # Must be a tensor

    return pos_weight


In [9]:
print(compute_pos_weight(train_d1, 'Profusion Label'))

tensor([0.4472])


### Higher Resolution Models


In [10]:
dicom_dir_1 = 'C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 1'
metadata_1 = pd.read_excel('C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 1/FileDatabaseWithRadiology.xlsx')
dicom_dir_2 = 'C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 2'
metadata_2 = pd.read_excel('C:/Users/user-pc/Masters/MSc - Project/MBOD_Datasets/Dataset 2/Database_Training-2024.08.28.xlsx')

d1 = classes.DICOMDataset1(dicom_dir=dicom_dir_1, metadata_df=metadata_1, target_size=224) 
d2 = classes.DICOMDataset2(dicom_dir=dicom_dir_2, metadata_df=metadata_2, target_size=224)

# Split datasets and store indices
train_indices_d1, val_indices_d1, test_indices_d1 = helpers.split_dataset(d1)
train_indices_d2, val_indices_d2, test_indices_d2 = helpers.split_dataset(d2)

# Save indices for later use
split_indices = {
    'd1': {'train': train_indices_d1, 'val': val_indices_d1, 'test': test_indices_d1},
    'd2': {'train': train_indices_d2, 'val': val_indices_d2, 'test': test_indices_d2}
}

label = 'Profusion'
d1.set_target(target_label=label, target_size=512)
d2.set_target(target_label=label, target_size=224)

train_d1 = Subset(d1, train_indices_d1)
val_d1 = Subset(d1, val_indices_d1)
test_d1 = Subset(d1, test_indices_d1)

train_d2 = Subset(d2, train_indices_d2)
val_d2 = Subset(d2, val_indices_d2)
test_d2 = Subset(d2, test_indices_d2)

In [18]:
model_512 = xrv.models.ResNet(weights="resnet50-res512-all")
model_224 = xrv.models.DenseNet(weights="densenet121-res224-all")


img, label = train_d1[12]
img_2, label_2 = train_d2[12]

print(img.shape, label)
print(img_2.shape, label_2)

torch.Size([1, 512, 512]) 0
torch.Size([1, 224, 224]) 1


In [19]:
x = model_224.features(img_2.unsqueeze(0))
print(x.shape)

torch.Size([1, 1024, 7, 7])


In [20]:
model_512.features2 = model_512.features

y = model_512.features2(img.unsqueeze(0))
print(y.shape)

torch.Size([1, 2048])


In [14]:
import classes
model_512.classifier = classes.BaseClassifier512(in_features=2048)

In [16]:
res = model_512.classifier(y)
print(res.shape)

torch.Size([1, 1])


In [6]:
from sklearn.metrics import cohen_kappa_score
labels = [True, False, True, True, False, True, False]
preds = [True, True, True, True, True, True, True]

print(cohen_kappa_score(labels, preds))

0.0
